Helm
----

![](https://helm.sh/img/chart-illustration.png)

Quelle: https://helm.sh/
- - - 

[Helm](https://helm.sh/) ist ein Werkzeug zur Verwaltung von Kubernetes-Charts. 

[Kubernetes-Charts](https://github.com/helm/charts) sind Pakete vorkonfigurierter Kubernetes-Ressourcen, welche wie Gesamthaft mittels eines Befehles installiert werden können

Helm Einsatzgebiete:
* Suchen und verwenden Sie beliebte Software, die als Kubernetes-Charts verpackt ist
* Teilen Sie Ihre eigenen Anwendungen als Kubernetes-Charts
* Erstellen Sie reproduzierbare Builds Ihrer Kubernetes-Anwendungen
* Verwalten Sie Ihre Kubernetes-Manifestdateien auf intelligente Weise
* Verwalten Sie Releases von Helm-Paketen

***
### Installation

Helm auf der Serverseite (tiller) ist in der [duk-demo](https://github.com/mc-b/duk-demo) Umgebung bereits installiert.  Ansonsten siehe Projekt [duk](https://github.com/mc-b/duk/tree/master/helm).

Wir brauchen noch den Client und müssen diesen Initialisieren:

In [1]:
! curl -s -L https://get.helm.sh/helm-v2.15.1-linux-amd64.tar.gz | bsdtar xvf - && mv linux-amd64/helm .; rm -rf linux-amd64
! ./helm init --client-only

x linux-amd64/
x linux-amd64/tiller
x linux-amd64/LICENSE
x linux-amd64/helm
x linux-amd64/README.md
$HELM_HOME has been configured at /home/jovyan/.helm.
Not installing Tiller due to 'client-only' flag having been set


***
### Services starten

Nun sind wir bereit und können Services, mittels [Kubernetes-Charts](https://github.com/helm/charts), installieren.

Beginnen wollen wir mit [Jenkins](https://jenkins.io/):

In [2]:
! ./helm install --name myjenkins stable/jenkins

NAME:   myjenkins
LAST DEPLOYED: Mon Oct 28 14:30:58 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME             DATA  AGE
myjenkins        5     2s
myjenkins-tests  1     2s

==> v1/Deployment
NAME       READY  UP-TO-DATE  AVAILABLE  AGE
myjenkins  0/1    1           0          1s

==> v1/PersistentVolumeClaim
NAME       STATUS  VOLUME         CAPACITY  ACCESS MODES  STORAGECLASS   AGE
myjenkins  Bound   rwo-volume-01  50Gi      RWO           local-storage  2s  Filesystem

==> v1/Pod(related)
NAME                        READY  STATUS    RESTARTS  AGE
myjenkins-5484fbf8f5-7nvx8  0/1    Init:0/1  0         1s

==> v1/Role
NAME                       AGE
myjenkins-schedule-agents  2s

==> v1/RoleBinding
NAME                       AGE
myjenkins-schedule-agents  1s

==> v1/Secret
NAME       TYPE    DATA  AGE
myjenkins  Opaque  2     2s

==> v1/Service
NAME             TYPE          CLUSTER-IP      EXTERNAL-IP  PORT(S)         AGE
myjenkins        LoadBalancer  10.

Im Gegensatz zum Recht einfachen Beispiel [Jenkins mit Blueocean](https://github.com/mc-b/duk/tree/master/devops#jenkins-mit-blueocean) wurden einiges mehr an Ressourcen erstellt u.a.:

* ConfigMap für die Konfiguration
* PersistenVolumeClaim zur Ablage der Daten
* Role und Service Account für Interation Agent - Jenkins?

Ausserdem wurde Password generiert. Diese und den UI-Port können wie wie folgt abfragen:

In [3]:
! echo "User    : admin"
! echo "Password: "$(kubectl get secret --namespace default myjenkins -o jsonpath="{.data.jenkins-admin-password}" | base64 --decode)
! echo "UI      : "$(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service myjenkins -o=jsonpath='{ .spec.ports[0].nodePort }')/")

User    : admin
Password: lbUVu5qbZV
UI      : http://192.168.137.100:31901


Wie bei [Jenkins mit Blueocean](https://github.com/mc-b/duk/tree/master/devops#jenkins-mit-blueocean) können wir mittels "neues Projekt" -> "Multibranch Pipeline" und Angabe eines Git Repositories, z.B. [https://github.com/mc-b/simple-java-maven-app](https://github.com/mc-b/simple-java-maven-app) im User Interface einen Job anlegen.

Wenn wir Jenkins nicht mehr brauchen, können wir den Service wieder löschen:


In [4]:
! ./helm delete --purge myjenkins

release "myjenkins" deleted
